In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/SkillFactory/DS_ML_data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
item_prieces = pd.read_csv(PATH+ '25_item_prices.csv' )

In [ ]:
item_prieces.head(3)

,item_id,item_price
0,338,1501
1,74,647
2,7696,825


In [ ]:
test_A = pd.read_csv(PATH+ '25_sample_a.csv' )
test_C = pd.read_csv(PATH+ '25_sample_c.csv' )
test_B = pd.read_csv(PATH+ '25_sample_b.csv' )


In [ ]:
test_A.head(3)

,user_id,item_id,action_id
0,84636,360,1
1,21217,9635,1
2,13445,8590,1


In [ ]:
test_C.head(3)

,user_id,item_id,action_id
0,274623,2863,1
1,265472,343,1
2,242779,6009,0


In [ ]:
test_B.head(3)

,user_id,item_id,action_id
0,118375,4105,1
1,107569,8204,1
2,175990,880,1


In [ ]:
def check_df(df):
  df_name =[x for x in globals() if globals()[x] is df][0]
  print('\n', df_name,'\n')
  print(df.info())
  print(' ')
  print(f'\n missed values: \n {df.isna().sum()}') 

In [ ]:
df_list = [test_A, test_B, test_C]
for dataset in df_list:
  check_df(dataset)


 test_A 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188912 entries, 0 to 1188911
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1188912 non-null  int64
 1   item_id    1188912 non-null  int64
 2   action_id  1188912 non-null  int64
dtypes: int64(3)
memory usage: 27.2 MB
None
 

 missed values: 
 user_id      0
item_id      0
action_id    0
dtype: int64

 test_B 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198438 entries, 0 to 1198437
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1198438 non-null  int64
 1   item_id    1198438 non-null  int64
 2   action_id  1198438 non-null  int64
dtypes: int64(3)
memory usage: 27.4 MB
None
 

 missed values: 
 user_id      0
item_id      0
action_id    0
dtype: int64

 test_C 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205510 entries, 0 to 1205509
Data columns (tot

In [ ]:
test_A['sample'] = 'A'
test_B['sample'] = 'B'
test_C['sample'] = 'C'
df = pd.concat([test_A, test_B, test_C])


In [ ]:
df.head(3)

,user_id,item_id,action_id,sample
0,84636,360,1,A
1,21217,9635,1,A
2,13445,8590,1,A


In [ ]:
df.shape

(3592860, 4)

In [ ]:
df = df.drop_duplicates(['user_id','item_id', 'action_id'], keep='last')
df = df.merge(item_prieces, on='item_id', how='left')

In [ ]:
df.shape

(3762000, 5)

In [ ]:
df.sample(10)

,user_id,item_id,action_id,sample,item_price
3507660,236709,3136,2,C,1692
3315846,285344,3375,1,C,424
2508966,243267,3997,1,C,1452
3459867,284447,6236,1,C,453
282416,50275,1052,1,A,1736
3149460,268920,4233,1,C,866
1009762,66800,7625,1,A,954
1380483,179894,2120,1,B,1163
3492636,214499,1861,1,C,382
3020462,281001,4358,0,C,1555


In [ ]:
df_A = df[df['sample'] == "A"]
df_B = df[df['sample'] == "B"]
df_C = df[df['sample'] == "C"]

In [ ]:
def groups_metric(df):
  new_df = {
        "user_id": [],
        "showed": [],
        "clicked": [],
        "purchased": [],
        "ctr": [],
        "purchase_rate": [],
        "gmv":  [],
        "sample": []

    }


  for user_id, group in df.groupby("user_id"):
      showed = len(group[group["action_id"] == 0])
      clicked = len(group[group["action_id"] == 1])
      bought = len(group[group["action_id"] == 2])
      gmv = group[group["action_id"] == 2]['item_price'].sum()
      sample = group["sample"].values[1]

      if len(group[group["action_id"] == 0].values) < 1 and (len(group[group["action_id"] == 1].values) < 1 or len(group[group["action_id"] == 2].values) <1):
        continue

      
      new_df["user_id"].append(user_id)
      new_df["showed"].append(showed)
      new_df["clicked"].append(clicked)
      new_df["purchased"].append(bought)

      new_df["ctr"].append(clicked/showed) #ctr (отношение кликов к просмотрам товаров)
      new_df["purchase_rate"].append(bought/showed) #purchase rate (отношение покупок к просмотрам товаров);
      new_df["gmv"].append(gmv) #gmv Gross Merchandise Value(оборот, сумма произведений количества покупок на стоимость покупки), где считаем 1 сессию за 1 точку (1 сессия на 1 пользователя)
      new_df["sample"].append(sample)

  return pd.DataFrame(new_df)

In [ ]:
pivot_A = groups_metric(df_A)
pivot_B = groups_metric(df_B)
pivot_C = groups_metric(df_C)

In [ ]:
pivot_ABC = pd.concat([pivot_A, pivot_B, pivot_C])
pivot_ABC.sample(3)

,user_id,showed,clicked,purchased,ctr,purchase_rate,gmv,sample
646,265574,216,1000,66,4.629630,0.305556,70309,C
768,78244,196,1000,44,5.102041,0.224490,39580,A
446,42961,209,1000,68,4.784689,0.325359,72918,A


In [ ]:
d = {'user_id':'user_amount', 'showed':'av_showed', 'clicked':'av_clicked', 'purchased':'av_purchased', 'ctr':'av_ctr', 'purchase_rate':'av_purchase_rate', 'gmv':'av_gmv' }
av_ABC = pivot_ABC.groupby('sample').agg({'user_id': 'count', 'showed': 'mean',	'clicked':'mean',	'purchased':'mean',	'ctr':'mean',	'purchase_rate':'mean',	'gmv':'mean'}).round(2)
av_ABC = av_ABC.rename(columns=d)
av_ABC

,user_amount,av_showed,av_clicked,av_purchased,av_ctr,av_purchase_rate,av_gmv
sample,,,,,,,
A,996,200.05,999.95,49.95,5.02,0.25,53273.27
B,996,159.93,999.96,99.87,6.29,0.63,106473.27
C,994,209.94,999.95,60.05,4.78,0.29,63998.90


In [ ]:
# Проведён тест равенства долей для A и C групп по всем метрикам.

# Сделан вывод.

import scipy as sp
from statsmodels.stats.proportion import proportions_ztest

print(proportions_ztest(
    count=[df_A.action_id.value_counts()[0],
           df_C.action_id.value_counts()[0]], 
    nobs=[df_A.action_id.value_counts()[1],
          df_C.action_id.value_counts()[1]])[1])

print(proportions_ztest(
    count=[df_A.action_id.value_counts()[2],
           df_C.action_id.value_counts()[2]], 
    nobs=[df_A.action_id.value_counts()[1],
          df_C.action_id.value_counts()[1]])[1])

print(sp.stats.ttest_ind(df_A.item_price.values, df_C.item_price.values))

7.030828728094025e-67
1.959683539900351e-214
Ttest_indResult(statistic=-0.29469603516347226, pvalue=0.7682261166268569)


In [ ]:
# Проведён тест равенства долей для A и B групп по всем метрикам.

# Сделан вывод


print(proportions_ztest(
    count=[df_A.action_id.value_counts()[0],
           df_B.action_id.value_counts()[0]], 
    nobs=[df_A.action_id.value_counts()[1],
          df_B.action_id.value_counts()[1]])[1])

print(proportions_ztest(
    count=[df_A.action_id.value_counts()[2],
           df_B.action_id.value_counts()[2]], 
    nobs=[df_A.action_id.value_counts()[1],
          df_B.action_id.value_counts()[1]])[1])

print(sp.stats.ttest_ind(df_A.item_price.values, df_B.item_price.values))




0.0
0.0
Ttest_indResult(statistic=-0.9403497249320629, pvalue=0.3470382922200962)


Имеются различия в метриках CTR и PR между выборками A и B, но не A и C.

Имеются различия в метрике GVM как между выборками A и B, так и A и C.